# Setup & Libraries

In [1]:
from torchtext import data, datasets
import torch
import spacy
import random
import numpy as np

def set_seed(seed = 0):
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed()

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils import clip_grad_norm_
import torch.nn as nn
from itertools import product
import pandas as pd
from collections import Counter, defaultdict

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import math, random

In [2]:
def load_artifacts():
    # Load previously saved artifacts from Part 1

    LOAD_PREPARED = True  
    if LOAD_PREPARED:
        bundle = torch.load("trec_artifacts.pt", map_location="cpu")


        TEXT = data.Field(**bundle["text_field_kwargs"])
        LABEL = data.LabelField(**bundle["label_field_kwargs"])

        fields = [("text", TEXT), ("label", LABEL)]

        def rebuild(dataset_blob):
            examples = [
                data.Example.fromlist([item["tokens"], item["label"]], fields)
                for item in dataset_blob
            ]
            return data.Dataset(examples, fields)

        train_data = rebuild(bundle["train_examples"])
        valid_data = rebuild(bundle["valid_examples"])
        test_data  = rebuild(bundle["test_examples"])

        for ds in (train_data, valid_data, test_data):
            ds.sort_key = lambda ex: len(ex.text)

        TEXT.build_vocab([])
        TEXT.vocab.itos = bundle["text_vocab_itos"]
        unk_token = TEXT.unk_token
        if unk_token not in TEXT.vocab.itos:
            raise ValueError("UNK token missing from serialized vocabulary.")
        unk_index = TEXT.vocab.itos.index(unk_token)
        TEXT.vocab.stoi = defaultdict(lambda: unk_index,
                                    {tok: i for i, tok in enumerate(TEXT.vocab.itos)})
        TEXT.vocab.vectors = bundle["text_vocab_vectors"]

        LABEL.build_vocab([])
        LABEL.vocab.itos = bundle["label_vocab_itos"]
        LABEL.vocab.stoi = {tok: i for i, tok in enumerate(LABEL.vocab.itos)}



        pretrained_embeddings = TEXT.vocab.vectors

        train_iter, valid_iter, test_iter = data.BucketIterator.splits(
            (train_data, valid_data, test_data),
            batch_size=bundle["batch_size"],
            sort_within_batch=True,
            device=DEVICE,
        )
        print("Artifacts loaded.")
        return (bundle, TEXT, LABEL,
                train_data, valid_data, test_data,
                train_iter, valid_iter, test_iter,
                pretrained_embeddings)
    

In [3]:
(bundle, TEXT, LABEL,
 train_data, valid_data, test_data,
 train_iter, valid_iter, test_iter,
 pretrained_embeddings) = load_artifacts()


Artifacts loaded.


## From 2e.
| Topic | Test Accuracy |
|-------|--------------|
| DESC  | 0.934783     |
| **ENTY**  | **0.648936**     |
| HUM   | 0.907692     |
| **ABBR**  | **0.333333**     |
| NUM   | 0.946903     |
| LOC   | 0.876543     |

## Weakest topics:
1. ABBR (33.3%): Small number of samples available in the training and testing data.
2. ENTY (64.9%): Diverse questions, overlapping patterns

## Design 1: Data Augmentation

In this strategy to tackle the scarcity of ABBR samples and the diverse wording of ENTY, more ABBR and ENTY samples are synthesised using Easy Data Augmentation (EDA) without external dependencies. This would expose the model to more instances of ABBR and ENTY samples during training. 

Upon exploration of the dataset, it has been noticed that ABBR samples only comprised of 1.5% of the training data. 
To reduce imbalance, we have taken a conservative approach to increase ABBR and ENTY data samples by 3  and 1.5 times respectively. In the training dataset, the number of ABBR data increased from 66 samples to 198 samples (3x) while the number of ENTY data samples increased from 984 samples to 1,476 samples (1.5x)

To ensure a fair evaluation, the RNN model of part 2 was retrained on the oversampled training dataset with the optimal hyperparameters obtained from part 2. Rather than continuing training from the previously saved model, retraining isolates the effect of oversampling and ensures that improvements can be attributed to the data balancing strategy rather than additional fine-tuning on a pre-trained model.

To test this design, the ClassifierRepresentationRNN model from (2e) with the optimal hyperparameters derived from Part 2 will be used and topic wise accuracy results will be compared to the ones in 2e. 

### Optimal Hyperparameters: 
- Learning Rate: 0.0001
- Optimizer: Adam
- Batch Size: 64
- Hidden Dimension: 256
- Dropout: 0.4
- Pooling: Max



In [4]:
label_counts = Counter([ex.label for ex in train_data.examples])
print("Before oversampling:", label_counts)

Before oversampling: Counter({'HUM': 1000, 'ENTY': 984, 'DESC': 906, 'NUM': 739, 'LOC': 667, 'ABBR': 66})


In [ ]:
#Transpose from [seq_len, batch] -> [batch, seq_len]
def extract_batch(batch):
    if isinstance(batch.text, (tuple, list)):
        text = batch.text[0] 
    else:
        text = batch.text
    
    labels = batch.label
    if text.dim() == 2 and text.size(0) != labels.size(0):
        text = text.transpose(0, 1)
    
    return text, labels

In [ ]:
BATCH_SIZE = 64

# IMPT: Rerun cell below after data augmentation to recreate the BucketIterator with the augmented data before training
# Recreate iterators with desired batch size
train_iter, valid_iter, test_iter = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    sort_within_batch=True,
    device=DEVICE,
)

print(f"Using BucketIterator with batch_size={BATCH_SIZE}")
print(f"Train batches: {len(train_iter)}")
print(f"Valid batches: {len(valid_iter)}")
print(f"Test batches: {len(test_iter)}")


Using BucketIterator with batch_size=64
Train batches: 73
Valid batches: 18
Test batches: 8


## Data Augmentation: Easy Data Augmentation (EDA) Methodology

EDA was applied to generate synthetic examples to increase sample size and diversity of training data, specifically ABBR and ENTY samples. The following EDA techniques were used: 

- Random Swap (rand_swap): Randomly select 2 tokens in a sentence and swap position for model to learn that position of words can vary without changing the overall meaning of the sample. 

- Random Deletion (rand_delete): Each token has a probability (p) of being deleted. This introduces noise or missing words that occurs in natural language. 

- Predefined templates: ENTY and ABBR templates are created with placeholders. New samples are gemerated using the structured question templates with the placeholder slots being filled with words from the dictionary of words created. 



In [18]:
def rand_swap(tokens, k=1):
    tokens = tokens[:] 
    L = len(tokens)
    for _ in range(k):
        if L < 2: break
        i, j = random.sample(range(L), 2)
        tokens[i], tokens[j] = tokens[j], tokens[i]
    return tokens

def rand_delete(tokens, p=0.1):
    keep = [t for t in tokens if random.random() > p]
    return keep if len(keep) >= 3 else tokens

def eda_augment(tokens, swaps=1, del_p=0.1):
    x = rand_swap(tokens, swaps)
    x = rand_delete(x, del_p)
    return x

# ---- ENTY templates (diversify “what/which” frames) ----
ENTY_TEMPLATES = [
    ["what", "is", "the", "{category}", "of", "{item}","?"],
    ["which", "{object}", "is", "known", "for", "{desc}","?"],
    ["what", "{thing}", "is", "used", "for", "{desc}","?"]
]
ENTY_SLOTS = {
    "category": ["type", "class", "category"],
    "thing": ["device", "vehicle", "instrument", "tool"],
    "item": ["quartz", "sushi", "python", "sahara", "euro",
          "mars", "oak", "bitcoin", "cello", "tulip"],
    "object": ["river", "planet", "currency", "instrument",
            "continent", "metal", "festival", "element"],
    "desc": ["navigation", "measuring time", "heat insulation", "data transfer",
            "communication", "transportation", "electric conduction"],

}

def make_enty_samples(n):
    outs = []
    for _ in range(n):
        tmpl = random.choice(ENTY_TEMPLATES)
        filled = []
        for tok in tmpl:
            if tok.startswith("{") and tok.endswith("}"):
                key = tok[1:-1]
                filled.append(random.choice(ENTY_SLOTS[key]))
            else:
                filled.append(tok)
        outs.append(filled)
    return outs

# ---- ABBR template augmentation ----
ABBR_BANK = [
    ("NASA", "National Aeronautics and Space Administration"),
    ("NATO", "North Atlantic Treaty Organization"),
    ("CPU", "Central Processing Unit"),
    ("GPU", "Graphics Processing Unit"),
    ("UNESCO", "United Nations Educational, Scientific and Cultural Organization"),
    ("FBI", "Federal Bureau of Investigation"),
    ("AI", "Artificial Intelligence"),
    ("EU", "European Union"),
]
ABBR_TEMPLATES = [
    ["what", "does", "{abbr}", "stand", "for","?"],
    ["what", "is", "the", "full", "form", "of", "{abbr}","?"],
    ["expand", "{abbr}","."],
    ["{abbr}", "stands", "for", "what","?"]
]

def make_abbr_samples(n):
    outs = []
    for _ in range(n):
        abbr, _full = random.choice(ABBR_BANK)
        tmpl = random.choice(ABBR_TEMPLATES)
        filled = [abbr if t == "{abbr}" else t for t in tmpl]
        filled = eda_augment(filled, swaps=1, del_p=0.0)
        outs.append(filled)
    return outs


In [19]:
fields_list = [("text", TEXT), ("label", LABEL)]
label_itos  = LABEL.vocab.itos
label_stoi  = LABEL.vocab.stoi

print("Before augmentation:", Counter([ex.label for ex in train_data.examples]))

ABBR_STR = "ABBR"
ENTY_STR = "ENTY"

enty_new_tokens = make_enty_samples(n=150) 
abbr_new_tokens = make_abbr_samples(n=66*2)

def tokens_to_examples(list_of_token_lists, label_str):
    examples = []
    for toks in list_of_token_lists:
        ex = data.Example.fromlist([toks, label_str], fields_list)
        examples.append(ex)
    return examples

enty_examples = tokens_to_examples(enty_new_tokens, ENTY_STR)
abbr_examples = tokens_to_examples(abbr_new_tokens, ABBR_STR)

train_data.examples.extend(enty_examples)
train_data.examples.extend(abbr_examples)
random.shuffle(train_data.examples)
train_data.sort_key = lambda ex: len(ex.text)

print("After augmentation :", Counter([ex.label for ex in train_data.examples]))

Before augmentation: Counter({'HUM': 1000, 'ENTY': 984, 'DESC': 906, 'NUM': 739, 'LOC': 667, 'ABBR': 66})
After augmentation : Counter({'ENTY': 1134, 'HUM': 1000, 'DESC': 906, 'NUM': 739, 'LOC': 667, 'ABBR': 198})


#### Classifier Representation RNN Model from Part 2 & Training Functions

In [7]:
class ClassifierRepresentationRNN(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, representation, dropout=0.0):
        super(ClassifierRepresentationRNN, self).__init__()
        num_embeddings, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=False)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, 6) # 6 possible labels
        self.representation = representation

    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded)

        # Conditional Logic to apply different representation techniques
        if self.representation.startswith('average_last_'):
            last_k = int(self.representation.split('_')[-1])
            k = min(last_k, output.size(1))
            rep = output[:, -k:, :].mean(dim=1)
        elif self.representation == 'max':
            rep, _ = torch.max(output, dim=1)
        elif self.representation == 'mean':
            rep = torch.mean(output, dim=1)
        elif self.representation == 'maxmean':
            max_pooled, _ = torch.max(output, dim=1)
            mean_pooled = torch.mean(output, dim=1)
            rep = (max_pooled + mean_pooled) / 2
        elif self.representation == 'sum':
            rep = torch.sum(output, dim=1)
        else:
            # Default: use last hidden state
            rep = hidden[-1]

        hidden = self.dropout(rep)
        out = self.fc(hidden)
        return out

In [8]:
# Define training, evaluation, and testing loops
# These work with BucketIterator (which returns batches with batch.text and batch.label)
def train_loop(model, iterator, optimizer, criterion, grad_clip=False, max_norm=1.0):
    model.train()
    total_loss, total_correct, total_examples = 0, 0, 0
    
    for batch in iterator:
        texts, labels = extract_batch(batch)
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()

        if grad_clip:
            clip_grad_norm_(model.parameters(), max_norm=max_norm)

        optimizer.step()
        total_loss += loss.item()
        total_correct += (outputs.argmax(1) == labels).sum().item()
        total_examples += labels.size(0)

    avg_loss = total_loss / len(iterator)
    avg_correct = total_correct / total_examples
    return avg_loss, avg_correct

def eval_loop(model, iterator, criterion):
    model.eval()
    total_loss, total_correct, total_examples = 0, 0, 0

    with torch.no_grad():
        for batch in iterator:
            texts, labels = extract_batch(batch)
            # texts and labels are already on DEVICE
            
            outputs = model(texts)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == labels).sum().item()
            total_examples += labels.size(0)

    avg_loss = total_loss / len(iterator)
    avg_correct = total_correct / total_examples
    return avg_loss, avg_correct

def test_loop(model, iterator):
    model.eval()
    total_correct, total_examples = 0, 0

    with torch.no_grad():
        for batch in iterator:
            texts, labels = extract_batch(batch)
            outputs = model(texts)
            total_correct += (outputs.argmax(1) == labels).sum().item()
            total_examples += labels.size(0)

    acc = total_correct / total_examples
    return acc


In [9]:
# Early stopper to prevent overfitting
class EarlyStopper:
    def __init__(self, patience=3, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.max_validation_acc = float('-inf')

    def early_stop(self, validation_acc):
        if validation_acc > self.max_validation_acc + self.min_delta:
            self.max_validation_acc = validation_acc
            self.counter = 0
        else:
            self.counter += 1
            return self.counter >= self.patience

In [10]:
# Function to capture metrics and print training results per epoch
def training_step(model, train_iter, valid_iter, optimizer, criterion, num_epochs, grad_clip=False, max_norm=1.0):
    train_losses, train_accuracies = [], []
    valid_losses, valid_accuracies = [], []
    early_stopper = EarlyStopper(patience=5, min_delta=0)

    for epoch in range(num_epochs):
        # Reinitialize epoch for BucketIterator
        train_iter.init_epoch()
        valid_iter.init_epoch()
        
        train_loss, train_acc = train_loop(model, train_iter, optimizer, criterion, grad_clip=grad_clip, max_norm=max_norm)
        valid_loss, valid_acc = eval_loop(model, valid_iter, criterion)

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        valid_losses.append(valid_loss)
        valid_accuracies.append(valid_acc)

        print(f"Epoch {epoch+1}:")
        print(f"Train loss: {train_loss:.4f}, Train acc: {train_acc:.4f}")
        print(f"Valid loss: {valid_loss:.4f}, Valid acc: {valid_acc:.4f}")

        if early_stopper.early_stop(valid_acc):
            print("Early Stopping Triggered! No Improvements to Validation Accuracy within Patience.")
            no_epochs = epoch+1
            break

        no_epochs = epoch+1

    return train_losses, train_accuracies, valid_losses, valid_accuracies, no_epochs


#### Hyperparameters Obtained from Hyperparameters Tuning in Part 2

In [11]:
# Set hyperparameters (best params from assignment)
LR = 0.0001
HIDDEN_DIM = 256
DROPOUT = 0.4
POOLING = 'max'
NUM_EPOCHS = 50  # Max epochs (early stopping will stop earlier if needed)

print(f"Hyperparameters:")
print(f"  Learning Rate: {LR}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Hidden Dimension: {HIDDEN_DIM}")
print(f"  Dropout: {DROPOUT}")
print(f"  Pooling: {POOLING}")
print(f"  Max Epochs: {NUM_EPOCHS}")

Hyperparameters:
  Learning Rate: 0.0001
  Batch Size: 64
  Hidden Dimension: 256
  Dropout: 0.4
  Pooling: max
  Max Epochs: 50


#### Model Training

In [ ]:
model = ClassifierRepresentationRNN(
    embedding_matrix=pretrained_embeddings.numpy(),
    hidden_dim=HIDDEN_DIM,
    representation=POOLING,
    dropout=DROPOUT
)
model = model.to(DEVICE)

# Initialize optimizer (Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# Loss function
criterion = nn.CrossEntropyLoss()

print(f"Model initialized on {DEVICE}")

Model initialized on cpu


In [22]:
# Train the model
train_losses, train_accuracies, valid_losses, valid_accuracies, epochs_ran = training_step(
    model=model,
    train_iter=train_iter,
    valid_iter=valid_iter,
    optimizer=optimizer,
    criterion=criterion,
    num_epochs=NUM_EPOCHS,
    grad_clip=False,
    max_norm=1.0
)

print(f"\nTraining completed after {epochs_ran} epochs")

Epoch 1:
Train loss: 1.7099, Train acc: 0.2444
Valid loss: 1.6172, Valid acc: 0.3119
Epoch 2:
Train loss: 1.5824, Train acc: 0.3773
Valid loss: 1.5426, Valid acc: 0.3523
Epoch 3:
Train loss: 1.4309, Train acc: 0.4694
Valid loss: 1.4315, Valid acc: 0.4284
Epoch 4:
Train loss: 1.2804, Train acc: 0.5332
Valid loss: 1.2771, Valid acc: 0.4982
Epoch 5:
Train loss: 1.1007, Train acc: 0.6156
Valid loss: 1.1308, Valid acc: 0.5881
Epoch 6:
Train loss: 0.8990, Train acc: 0.7145
Valid loss: 0.8357, Valid acc: 0.7284
Epoch 7:
Train loss: 0.7167, Train acc: 0.7903
Valid loss: 0.7358, Valid acc: 0.7615
Epoch 8:
Train loss: 0.5958, Train acc: 0.8280
Valid loss: 0.6244, Valid acc: 0.8018
Epoch 9:
Train loss: 0.5198, Train acc: 0.8559
Valid loss: 0.5852, Valid acc: 0.8064
Epoch 10:
Train loss: 0.4863, Train acc: 0.8620
Valid loss: 0.6237, Valid acc: 0.7615
Epoch 11:
Train loss: 0.4319, Train acc: 0.8764
Valid loss: 0.5360, Valid acc: 0.8193
Epoch 12:
Train loss: 0.3774, Train acc: 0.8951
Valid loss: 0.5

In [16]:
# Define function for per-topic accuracy evaluation
def topic_eval_loop_rnn(model, iterator, label_vocab, device):
    model.eval()
    n = len(label_vocab.itos)
    correct = [0] * n
    total = [0] * n
    
    with torch.no_grad():
        for batch in iterator:
            texts, labels = extract_batch(batch)
            outputs = model(texts)
            preds = torch.argmax(outputs, dim=1)
            
            for p, y in zip(preds.tolist(), labels.tolist()):
                total[y] += 1
                correct[y] += int(p == y)
    
    per_topic = {}
    for idx, lab in enumerate(label_vocab.itos):
        per_topic[lab] = float('nan') if total[idx] == 0 else correct[idx] / total[idx]
    
    return per_topic


In [ ]:
topic_acc = topic_eval_loop_rnn(model, test_iter, LABEL.vocab, device=DEVICE)
print("Topic-wise accuracy for ABBR and ENTY Samples:")
for k, v in topic_acc.items():
    if k in ("ABBR", "ENTY"):
        print(f"  {k}: {v:.4f}")

Topic-wise accuracy for ABBR and ENTY Samples:
  ENTY: 0.7128
  ABBR: 1.0000


ENTY: From 65% to 71%
ABBR: From 33% to 100%
Mention that ABBR test is only 9 samples -> 1 extra correct prediction will bring about a more than 10% increase in accuracy rate

After applying EDA techniques, there was a significant improvement in topic-wise accuracy for ABBR and ENTY samples. The accuracy rate for ENTY increased from 64.89% to 71.28% while for ABBR, accuracy increased from 33.33% to 100%. 

In the case ENTY class, this improvement shows the effectiveness of EDA in helping the model learn the linguistic nuances associated with ENTY recognition. As for ABBR, the significant increase in accuracy rate shows that the lack of samples in the original data was a limiting factor during model training. However, it is important to interpret this result with caution as the test set only contained 9 ABBR samples. Therefore, a single additional correct prediction can lead to a more than 10% increase in accuracy. Nonetheless, this result still highlights that the effectiveness of using EDA to enhance the model's understanding of the ABBR samples, which were previously underrepresented. 

# Design 2: 

## Model-side Strategy: Class-weighted Loss

In the original TREC dataset, there is a strong class imbalance in categories such as ABBR, causing the trained model to perform poorly in topic-specific accuracy rates as the model would learn to be biased towards majority classes. 

Instead of modifying the dataset, a model focused strategy would be to make the loss function cost sensitive by introducing class-weighted loss. This would mean that misclassifying rare classes will incur a larger penalty than if a majority class is misclassified. 

To achieve this, class weights are introduced to the loss function. Class weights are inversely proportional to its frequency. The rest of the model architecture and hyperparameters from Part 2 remain unchanged, ensuring a fair comparison with the base RNN model used in 2e. 


In [ ]:
# Reload original data (without data augmentation)
(bundle, TEXT, LABEL,
 train_data, valid_data, test_data,
 train_iter, valid_iter, test_iter,
 pretrained_embeddings) = load_artifacts()

Artifacts loaded.


In [28]:
BATCH_SIZE = 64

train_iter, valid_iter, test_iter = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    sort_within_batch=True,
    device=DEVICE,
)

print(f"Using BucketIterator with batch_size={BATCH_SIZE}")
print(f"Train batches: {len(train_iter)}")
print(f"Valid batches: {len(valid_iter)}")
print(f"Test batches: {len(test_iter)}")

Using BucketIterator with batch_size=64
Train batches: 69
Valid batches: 18
Test batches: 8


In [29]:
# collect training labels as indices
train_label_ids = [LABEL.vocab.stoi[ex.label] for ex in train_data.examples]

counts = Counter(train_label_ids)
num_classes = len(LABEL.vocab)
total = sum(counts.values())

# inverse-frequency weights
weights = []
for i in range(num_classes):
    w = total / max(1, counts[i])
    weights.append(w)

class_weights = torch.tensor(weights, dtype=torch.float).to(DEVICE)
print("Class weights:", class_weights)

criterion_weighted = nn.CrossEntropyLoss(weight=class_weights)

Class weights: tensor([ 4.3620,  4.4329,  4.8146,  5.9026,  6.5397, 66.0909])


In [31]:
model_w = ClassifierRepresentationRNN(
    embedding_matrix=TEXT.vocab.vectors,
    hidden_dim=HIDDEN_DIM,
    representation=POOLING,
    dropout=0.4
).to(DEVICE)

optimizer = torch.optim.Adam(model_w.parameters(), lr=1e-4)

train_losses, train_accuracies, valid_losses, valid_accuracies, epochs_ran = training_step(
    model=model_w,
    train_iter=train_iter,
    valid_iter=valid_iter,
    optimizer=optimizer,
    criterion=criterion,
    num_epochs=NUM_EPOCHS,
    grad_clip=False,
    max_norm=1.0
)

print(f"\nTraining completed after {epochs_ran} epochs")

/var/folders/bw/_dd4c0bn1ws10nbrwf6ls6hc0000gn/T/ipykernel_57925/3286407227.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=False)


Epoch 1:
Train loss: 1.6921, Train acc: 0.2575
Valid loss: 1.6139, Valid acc: 0.3248
Epoch 2:
Train loss: 1.5642, Train acc: 0.3693
Valid loss: 1.5421, Valid acc: 0.3798
Epoch 3:
Train loss: 1.4493, Train acc: 0.4702
Valid loss: 1.4162, Valid acc: 0.5330
Epoch 4:
Train loss: 1.2750, Train acc: 0.5614
Valid loss: 1.2577, Valid acc: 0.5844
Epoch 5:
Train loss: 1.0689, Train acc: 0.6421
Valid loss: 1.0355, Valid acc: 0.6615
Epoch 6:
Train loss: 0.8532, Train acc: 0.7352
Valid loss: 0.8459, Valid acc: 0.7349
Epoch 7:
Train loss: 0.7097, Train acc: 0.7811
Valid loss: 0.7545, Valid acc: 0.7450
Epoch 8:
Train loss: 0.6451, Train acc: 0.8058
Valid loss: 0.6981, Valid acc: 0.7761
Epoch 9:
Train loss: 0.5484, Train acc: 0.8349
Valid loss: 0.6880, Valid acc: 0.7661
Epoch 10:
Train loss: 0.5004, Train acc: 0.8501
Valid loss: 0.6280, Valid acc: 0.7890
Epoch 11:
Train loss: 0.4647, Train acc: 0.8620
Valid loss: 0.6099, Valid acc: 0.7991
Epoch 12:
Train loss: 0.4186, Train acc: 0.8778
Valid loss: 0.6

In [32]:
topic_acc = topic_eval_loop_rnn(model_w, test_iter, LABEL.vocab, device=DEVICE)
print("Topic-wise accuracy for ABBR and ENTY Samples:")
for k, v in topic_acc.items():
    if k in ("ABBR", "ENTY"):
        print(f"  {k}: {v:.4f}")

Topic-wise accuracy for ABBR and ENTY Samples:
  ENTY: 0.7021
  ABBR: 0.6667


After applying the class-weighted cross-entropy loss, the model had a noticeable improvement in the accuracy of ABBR and ENTY, which were the weaker topics. The accuracy rate of ENTY and ABBR increased from 64.89% to 70.21% and 33.33% to 66.67%. 

Although class-weighted loss mainly targets highly underrepresented classes, ENTY still benefitted from this strategy as ENTY occurs less frequently than the dominant categories. On the other hand, for ABBR, the improvement was more significant, with its accuracy rate doubling. This highlights that the cost-sensitive training was effective in addressing the data imbalance. Hence, class-weighted loss is a practical approach to enhance performance of models when it comes to underrepresented classes without modifying the dataset. 


# Design 3:

# Model-side Strategy: Multi-Task RNN: 

Rather than training separate models for each task, a multi-task RNN is trained to perform multiple related tasks simultaneously. In this model architecure, the model performs 2 related classification tasks simultaneously: 
1. Main task: Predict the topic label
2. Auxiliary Task: A simpler task that helps the main task generalise better. 
Since we are targeting the topics ABBR and ENTY, the auxiliary task is designed to make the model more aware of these categories. 

#### Auxiliary Tasks:

Each sample can have additional binary indicators: 
- is_ABBR = 1 if the label is ABBR, otherwise = 0
- is_ENTY = 1 if the label is ENTY, otherwise = 0

During training of the model, the model learns these 3 objectives simultaneously: 
- Main: Predict topic 
- Auxiliary: Predict is_ABBR
- Auxiliary: Predict is_ENTY

Since main task is a multi-class classification, cross entropy loss is used. As for the auxiliary tasks, they are binary classification, thus binary cross entropy loss is used. During training, the gradients from all 3 tasks jointly updates the shared encoder as the total loss is a weighted combination of all 3 tasks. 

In [33]:
class ClassifierRNNMultiTask(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, representation="max", dropout=0.4, num_classes=6):
        super().__init__()
        num_embeddings, embedding_dim = embedding_matrix.shape
        
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float),
            freeze=False
        )
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        
        self.representation = representation
        
        # main classifier
        self.fc_main = nn.Linear(hidden_dim, num_classes)
        
        # auxiliary heads: is ABBR? is ENTY?
        self.fc_abbr = nn.Linear(hidden_dim, 1)
        self.fc_enty = nn.Linear(hidden_dim, 1)

    def encode(self, x):
        emb = self.embedding(x)             # [B, L, E]
        output, hidden = self.rnn(emb)      # output: [B, L, H]
        
        if self.representation == "mean":
            rep = output.mean(dim=1)
        elif self.representation == "max":
            rep, _ = output.max(dim=1)
        else:
            rep = hidden[-1]                # last hidden
        
        rep = self.dropout(rep)
        return rep

    def forward(self, x):
        rep = self.encode(x)                # [B, H]
        logits_main = self.fc_main(rep)     # [B, C]
        logit_abbr = self.fc_abbr(rep).squeeze(-1)  # [B]
        logit_enty = self.fc_enty(rep).squeeze(-1)  # [B]
        return logits_main, logit_abbr, logit_enty


In [34]:
abbr_idx = LABEL.vocab.stoi["ABBR"]
enty_idx = LABEL.vocab.stoi["ENTY"]

bce = nn.BCEWithLogitsLoss()

def multitask_loss(logits_main, logit_abbr, logit_enty, targets):
    loss_main = criterion_ce(logits_main, targets)
    
    # build binary targets
    target_abbr = (targets == abbr_idx).float()
    target_enty = (targets == enty_idx).float()
    
    loss_abbr = bce(logit_abbr, target_abbr)
    loss_enty = bce(logit_enty, target_enty)
    
    # weight auxiliaries (tune lambdas; start small)
    return loss_main + 0.5 * loss_abbr + 0.5 * loss_enty


In [35]:
model_mt = ClassifierRNNMultiTask(
    embedding_matrix=TEXT.vocab.vectors,
    hidden_dim=HIDDEN_DIM,
    representation=POOLING,   # using 'max' pooling
    dropout=DROPOUT,
    num_classes=len(LABEL.vocab)
).to(DEVICE)

criterion_ce = nn.CrossEntropyLoss()
bce = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_mt.parameters(), lr=LR)

/var/folders/bw/_dd4c0bn1ws10nbrwf6ls6hc0000gn/T/ipykernel_57925/1378855307.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(embedding_matrix, dtype=torch.float),


In [42]:
def train_multitask_rnn(
    model,
    train_iter,
    valid_iter,
    optimizer,
    num_epochs,
    device,
    patience=5,
    grad_clip=True,
    max_norm=1.0,
    save_path="rnn_multitask_best.pt"
):
    best_val_acc = -1.0
    wait = 0

    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": []
    }

    for epoch in range(1, num_epochs + 1):
        # ---- TRAIN ----
        model.train()
        total_loss = 0.0
        total_correct = 0
        total_examples = 0
        num_batches = 0

        for batch in train_iter:
            texts, labels = extract_batch_multitaskRNN(batch, device)

            optimizer.zero_grad()
            logits_main, logit_abbr, logit_enty = model(texts)

            loss = multitask_loss(logits_main, logit_abbr, logit_enty, labels)
            loss.backward()

            if grad_clip:
                clip_grad_norm_(model.parameters(), max_norm=max_norm)

            optimizer.step()

            total_loss += loss.item()
            preds = logits_main.argmax(dim=1)
            total_correct += (preds == labels).sum().item()
            total_examples += labels.size(0)
            num_batches += 1

        train_loss = total_loss / max(1, num_batches)
        train_acc = total_correct / max(1, total_examples)

        # ---- VALIDATION (main task only) ----
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_examples = 0
        val_batches = 0

        with torch.no_grad():
            for batch in valid_iter:
                texts, labels = extract_batch_multitaskRNN(batch, device)
                logits_main, logit_abbr, logit_enty = model(texts)

                loss_main = criterion_ce(logits_main, labels)
                val_loss += loss_main.item()

                preds = logits_main.argmax(dim=1)
                val_correct += (preds == labels).sum().item()
                val_examples += labels.size(0)
                val_batches += 1

        val_loss /= max(1, val_batches)
        val_acc = val_correct / max(1, val_examples)

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(
            f"Epoch {epoch:02d} | "
            f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
            f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
        )

        # ---- EARLY STOPPING ----
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            wait = 0
            torch.save(model.state_dict(), save_path)
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping triggered.")
                break

    return history, best_val_acc


In [40]:
def extract_batch_multitaskRNN(batch, device):
    # batch.text can be [seq_len, batch] or (text, lengths)
    if isinstance(batch.text, (tuple, list)):
        text = batch.text[0]
    else:
        text = batch.text

    labels = batch.label

    # If shape is [seq_len, batch], transpose to [batch, seq_len]
    if text.dim() == 2 and text.size(0) != labels.size(0):
        text = text.transpose(0, 1)

    return text.to(device), labels.to(device)


In [43]:
model_mt = ClassifierRNNMultiTask(
    embedding_matrix=TEXT.vocab.vectors,
    hidden_dim=HIDDEN_DIM,
    representation=POOLING,   # 'max' (from Part 2)
    dropout=DROPOUT,
    num_classes=len(LABEL.vocab)
).to(DEVICE)

optimizer = torch.optim.Adam(model_mt.parameters(), lr=LR)

history_mt, best_val_acc_mt = train_multitask_rnn(
    model=model_mt,
    train_iter=train_iter,
    valid_iter=valid_iter,
    optimizer=optimizer,
    num_epochs=NUM_EPOCHS,
    device=DEVICE,
    patience=5,
    grad_clip=True,
    max_norm=1.0,
    save_path="rnn_multitask_best.pt"
)

print("Best Val Acc (Multi-task):", best_val_acc_mt)


/var/folders/bw/_dd4c0bn1ws10nbrwf6ls6hc0000gn/T/ipykernel_57925/1378855307.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(embedding_matrix, dtype=torch.float),


Epoch 01 | Train Loss: 2.1760 | Train Acc: 0.2455 | Val Loss: 1.6100 | Val Acc: 0.3000
Epoch 02 | Train Loss: 1.9148 | Train Acc: 0.3306 | Val Loss: 1.5353 | Val Acc: 0.4743
Epoch 03 | Train Loss: 1.8004 | Train Acc: 0.4477 | Val Loss: 1.4389 | Val Acc: 0.4862
Epoch 04 | Train Loss: 1.6548 | Train Acc: 0.5142 | Val Loss: 1.3450 | Val Acc: 0.5229
Epoch 05 | Train Loss: 1.4983 | Train Acc: 0.5830 | Val Loss: 1.1655 | Val Acc: 0.6211
Epoch 06 | Train Loss: 1.3002 | Train Acc: 0.6637 | Val Loss: 1.0045 | Val Acc: 0.6927
Epoch 07 | Train Loss: 1.0582 | Train Acc: 0.7460 | Val Loss: 0.8501 | Val Acc: 0.7220
Epoch 08 | Train Loss: 0.8931 | Train Acc: 0.7886 | Val Loss: 0.8011 | Val Acc: 0.7376
Epoch 09 | Train Loss: 0.7985 | Train Acc: 0.8150 | Val Loss: 0.6327 | Val Acc: 0.7807
Epoch 10 | Train Loss: 0.7070 | Train Acc: 0.8386 | Val Loss: 0.7170 | Val Acc: 0.7633
Epoch 11 | Train Loss: 0.6575 | Train Acc: 0.8528 | Val Loss: 0.6710 | Val Acc: 0.7954
Epoch 12 | Train Loss: 0.5965 | Train Acc: 

In [46]:
total_correct = 0
total_examples = 0

# Reload best model for evaluation
best_mt = ClassifierRNNMultiTask(
    embedding_matrix=TEXT.vocab.vectors,
    hidden_dim=HIDDEN_DIM,
    representation=POOLING,
    dropout=DROPOUT,
    num_classes=len(LABEL.vocab)
).to(DEVICE)

best_mt.load_state_dict(torch.load("rnn_multitask_best.pt", map_location=DEVICE))
best_mt.eval()

with torch.no_grad():
    for batch in test_iter:
        texts, labels = extract_batch_multitaskRNN(batch, DEVICE)
        logits_main, logit_abbr, logit_enty = best_mt(texts)
        preds = logits_main.argmax(dim=1)
        total_correct += (preds == labels).sum().item()
        total_examples += labels.size(0)

test_acc = total_correct / max(1, total_examples)
print(f"Test Accuracy (Multi-task RNN): {test_acc:.4f}")

Test Accuracy (Multi-task RNN): 0.9080


/var/folders/bw/_dd4c0bn1ws10nbrwf6ls6hc0000gn/T/ipykernel_57925/1378855307.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(embedding_matrix, dtype=torch.float),


In [50]:
correct = defaultdict(int)
total = defaultdict(int)

with torch.no_grad():
    for batch in test_iter:
        texts, labels = extract_batch_multitaskRNN(batch, DEVICE)
        logits_main, _, _ = best_mt(texts)
        preds = logits_main.argmax(dim=1)

        for p, y in zip(preds.tolist(), labels.tolist()):
            lab = LABEL.vocab.itos[y]
            total[lab] += 1
            if p == y:
                correct[lab] += 1

print("Topic-wise Test Accuracy (Multi-task RNN):")
for lab in LABEL.vocab.itos:
    if total[lab] > 0:
        acc = correct[lab] / total[lab]
        if lab in ("ABBR", "ENTY"):
            print(f"{lab:>5}: {acc:.4f}")


Topic-wise Test Accuracy (Multi-task RNN):
 ENTY: 0.8191
 ABBR: 0.7778


The multi-task RNN with auxiliary binary classification heads (is_ABBR and is_ENTY) demonstrated significant improvement. The topic-wise test accuracy increased to 81.91% for ENTY and 77.78% for ABBR, compared to the baseline accuracy of 64.89% and 33.33% for ENTY and ABBR respectively. 

The improvement for the topic ENTY suggests that the auxiliary heads helped the model to disambiguate ENTY-type questions, which often overlap in structure with other categories due to its diverse phrasing. As for ABBR, the muti-task RNN helped to reduce the effect of data imbalance, allowing model to be more sensitive to the underrepressented class. 